In [122]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
transactions = pd.read_csv("data_with_groups.csv")

In [6]:
transactions.head(10)

,Unnamed: 0,OBJECTID,AGENCY,TRANSACTION_DATE,TRANSACTION_AMOUNT,VENDOR_NAME,VENDOR_STATE_PROVINCE,MCC_DESCRIPTION,join_key,MCC,REQUIRED NAME IN AUTHORIZATION REQUEST / CLEARING RECORD,GROUP,MONTH_YEAR
0,0,2,Department of Mental Health,2009/01/05 05:00:00+00,229.50,WW GRAINGER 912,DC,industrialsuppliesnotelsewhereclassified,industrialsuppliesnotelsewhereclassified,5085.0,NaN,Retail outlets,2009/01
1,1,3,District Department of Transportation,2009/01/05 05:00:00+00,3147.33,BRANCH SUPPLY,DC,stationeryofficeandschoolsupplystores,stationerystoresofficeandschoolsupplystores,5943.0,NaN,Miscellaneous outlets,2009/01
2,2,4,Department of Youth Rehabilitation Services,2009/01/05 05:00:00+00,54.00,VIDEO MONITORING SRVCS,NY,motionpicturesvideotapeproductionanddistribution,motionpictureandvideotapeproductionanddistribu...,7829.0,NaN,Amusement and entertainment,2009/01
3,3,5,Office of Employee Appeals,2009/01/05 05:00:00+00,137.35,STAPLES 00102186,DC,stationeryofficesuppliesprintingwritingpaper,stationeryofficesuppliesprintingandwritingpaper,5111.0,NaN,Retail outlets,2009/01
4,4,6,Office of Police Complaints,2009/01/06 05:00:00+00,1340.80,FAITH MGMT CONSULTING,DC,stationeryofficeandschoolsupplystores,stationerystoresofficeandschoolsupplystores,5943.0,NaN,Miscellaneous outlets,2009/01
5,5,7,Fire & Emergency Medical Services,2009/01/06 05:00:00+00,667.00,"20/20 GENESYSTEMS, INC",MD,businessservicesnotelsewhereclassified,businessservicesnotelsewhereclassified,7399.0,NaN,Business services,2009/01
6,6,8,Fire & Emergency Medical Services,2009/01/06 05:00:00+00,2760.00,FEDERAL RESOURCES SUPP,MD,professionalservicesnotelsewheredefined,professionalservicesnotelsewhereclassified,8999.0,NaN,Professional services and membership organizat...,2009/01
7,7,9,Fire & Emergency Medical Services,2009/01/06 05:00:00+00,190.42,WW GRAINGER 912,DC,industrialsuppliesnotelsewhereclassified,industrialsuppliesnotelsewhereclassified,5085.0,NaN,Retail outlets,2009/01
8,8,10,Fire & Emergency Medical Services,2009/01/06 05:00:00+00,34.89,WW GRAINGER 912,DC,industrialsuppliesnotelsewhereclassified,industrialsuppliesnotelsewhereclassified,5085.0,NaN,Retail outlets,2009/01
9,9,11,Fire & Emergency Medical Services,2009/01/06 05:00:00+00,72.36,WW GRAINGER 912,DC,industrialsuppliesnotelsewhereclassified,industrialsuppliesnotelsewhereclassified,5085.0,NaN,Retail outlets,2009/01


In [7]:
list_of_month_year = []
for x in transactions["TRANSACTION_DATE"]:
    list_of_month_year.append(x[0:7])
    

transactions["MONTH_YEAR"] = list_of_month_year

In [24]:
def median_costs_score(category_df, agency, agency_list):
    
    # Get median cost per purchase
    median_agency = np.median(category_df[category_df["AGENCY"] == agency]["TRANSACTION_AMOUNT"])
    median_costs = np.array([median_agency])
    
    for agency_b in agency_list: 
        median_agency_b = np.median(category_df[category_df["AGENCY"] == agency_b]["TRANSACTION_AMOUNT"])
        median_costs = np.append(median_costs, median_agency_b)
        
    
    if max(median_costs) == min(median_costs):
        return 1
    
    median_costs_mod = (median_costs - np.min(median_costs))/(np.max(median_costs) - np.min(median_costs))
    
    distances = np.abs(median_costs_mod[0] - median_costs_mod)
    
    return 1 - distances[1:]
    

In [48]:
def sum_costs_score(category_df, agency, agency_list):
    
    # Get total cost paid
    sum_agency = np.sum(category_df[category_df["AGENCY"] == agency]["TRANSACTION_AMOUNT"])
    sum_costs = np.array([sum_agency])
    
    for agency_b in agency_list: 
        sum_agency_b = np.sum(category_df[category_df["AGENCY"] == agency_b]["TRANSACTION_AMOUNT"])
        sum_costs = np.append(sum_costs, sum_agency_b)
        
    if max(sum_costs) == min(sum_costs):
        return 1
      
    sum_costs_mod = (sum_costs - np.min(sum_costs))/(np.max(sum_costs) - np.min(sum_costs))
    
    distances = np.abs(sum_costs_mod[0] - sum_costs_mod)
    
    return 1 - distances[1:]
    

In [42]:
def median_per_month(category_df, agency, agency_list):
    
    # Get median payment per month
    full_agency = np.append(agency_list, agency)
    
    subset_companies = category_df[category_df["AGENCY"].isin(full_agency)]
    median_transactions = category_df.groupby(["AGENCY", "MONTH_YEAR"], as_index=False).agg({"TRANSACTION_AMOUNT": "sum"}).groupby("AGENCY", as_index=False).agg({"TRANSACTION_AMOUNT": "median"})
    median_agency = median_transactions[median_transactions["AGENCY"] == agency].iloc[0][1]
    median_per_month_costs = np.array([median_agency])
    for agency_b in agency_list:
        median_agency_b = median_transactions[median_transactions["AGENCY"] == agency_b].iloc[0][1]
        median_per_month_costs = np.append(median_per_month_costs, median_agency_b)
      
    if max(median_per_month_costs) == min(median_per_month_costs):
        return 1
      
    median_per_month_costs_mod = (median_per_month_costs - np.min(median_per_month_costs))/(np.max(median_per_month_costs) - np.min(median_per_month_costs))
    
    distances = np.abs(median_per_month_costs_mod[0] - median_per_month_costs_mod)
    
    return 1 - distances[1:]

In [43]:
def median_orders_per_month(category_df, agency, agency_list):
    
    # Get median orders per month
    full_agency = np.append(agency_list, agency)
    subset_companies = category_df[category_df["AGENCY"].isin(full_agency)]
    median_transactions = category_df.groupby(["AGENCY", "MONTH_YEAR"], as_index=False).agg({"TRANSACTION_AMOUNT": "count"}).groupby("AGENCY", as_index=False).agg({"TRANSACTION_AMOUNT": "median"})
    median_agency = median_transactions[median_transactions["AGENCY"] == agency].iloc[0][1]
    
    median_per_month_count = np.array([median_agency])
    
    for agency_b in agency_list:
        median_agency_b = median_transactions[median_transactions["AGENCY"] == agency_b].iloc[0][1]
        median_per_month_count = np.append(median_per_month_count, median_agency_b)
     
    if max(median_per_month_count) == min(median_per_month_count):
        return 1
      
    median_per_month_count_mod = (median_per_month_count - np.min(median_per_month_count))/(np.max(median_per_month_count) - np.min(median_per_month_count))
    
    distances = np.abs(median_per_month_count_mod[0] - median_per_month_count_mod)
    
    return 1 - distances[1:]
    

In [55]:
def count_orders(category_df, agency, agency_list):
    
    # Get number of orders
    count_agency = category_df[category_df["AGENCY"] == agency].shape[0]
    count_orders_array = np.array([count_agency])
    
    for agency_b in agency_list: 
        count_agency_b = category_df[category_df["AGENCY"] == agency_b].shape[0]
        count_orders_array = np.append(count_orders_array, count_agency_b)
        
    if max(count_orders_array) == min(count_orders_array):
        return 1
    
    count_orders_array_mod = (count_orders_array - np.min(count_orders_array))/(np.max(count_orders_array) - np.min(count_orders_array))
    
    distances = np.abs(count_orders_array_mod[0] - count_orders_array_mod)
    
    return 1 - distances[1:]
    

In [202]:
def percent_transaction_amount(agency, agency_list, transactions_pct_new):
    
    # Get percent transactions
    
    pct_trans_agency = transactions_pct_new[transactions_pct_new["AGENCY"] == agency].iloc[0][2]
    pct_trans_array = np.array([pct_trans_agency])
    
    for agency_b in agency_list: 
        pct_trans_agency_b = transactions_pct_new[transactions_pct_new["AGENCY"] == agency_b].iloc[0][2]
        pct_trans_array = np.append(pct_trans_array, pct_trans_agency_b)
        
    
    distances = np.abs(pct_trans_array[0] - pct_trans_array)
    
    return 1 - distances[1:]
    

In [83]:
def find_similarity_scores(category_df, agency, agency_list):
    
    # Get sim scores
    
    median_costs = median_costs_score(category_df, agency, agency_list)
    sum_costs = sum_costs_score(category_df, agency, agency_list)
    median_per_month_costs = median_per_month(category_df, agency, agency_list)
    median_per_month_count = median_orders_per_month(category_df, agency, agency_list)
    count_orders_array = count_orders(category_df, agency, agency_list)
    sim_scores = (median_costs + sum_costs + median_per_month_costs + median_per_month_count + count_orders_array)/5
    return sim_scores

In [84]:
def sim_scores(category_df, agency, agency_list, count_agency):
    
    # Concat results
    agency_list = np.delete(agency_list, count_agency)
    
    sim_scores = find_similarity_scores(category_df, agency, agency_list)
    
    sim_scores = np.concatenate([sim_scores[0:count_agency], np.array([1]), sim_scores[count_agency:]])
    return sim_scores

In [88]:
def get_sim_matrices(transactions):
    
    
    # make matrices
    group_list = np.unique(list(transactions["GROUP"]))
    final_dict = {}
    
    for group in group_list:
        print(group)
        category_df = transactions[transactions["GROUP"] == group]
        agency_list = np.unique(category_df["AGENCY"])
        agency_matrix = np.zeros((len(agency_list), len(agency_list)))
        count_agency = 0
        for agency in agency_list:
            sim = sim_scores(category_df, agency, agency_list, count_agency)
            agency_matrix[count_agency, ] = sim
            count_agency += 1
        agency_dict = dict(zip(range(len(agency_list)), agency_list))
        final_dict[group] = pd.DataFrame(agency_matrix).rename(columns = agency_dict, index = agency_dict)
        
        
    return final_dict   


final_dict = get_sim_matrices(transactions)

Agricultural services
Airlines
Amusement and entertainment
Business services
Car rental
Clothing outlets
Contracted services
Government services
Lodging
Miscellaneous outlets
Professional services and membership organizations
Repair services
Retail outlets
Service providers
Transportation
Utilities
nan


In [203]:
def get_new_matrix(transactions):
    
    
    # Add sixth metric
    transactions_sales = transactions.groupby(['AGENCY', 'GROUP']).agg({'TRANSACTION_AMOUNT': 'sum'})

    transactions_pcts = transactions_sales.groupby(level=0).apply(lambda x:
                                                 x / float(x.sum()))
    
    transactions_pcts = transactions_pcts.reset_index()
    
    group_list = np.unique(list(transactions["GROUP"]))
    new_final_dict = {}
    
    for group in group_list:
        print(group)
        transactions_pct_new = transactions_pcts[transactions_pcts["GROUP"] == group]
        agency_list = np.unique(transactions_pct_new["AGENCY"])
        agency_matrix = np.zeros((len(agency_list), len(agency_list)))
        count_agency = 0
        for agency in agency_list:
            new_agency_list = np.delete(agency_list, count_agency)
            sim_01 = percent_transaction_amount(agency, new_agency_list, transactions_pct_new)
            sim = np.concatenate([sim_01[0:count_agency], np.array([1]), sim_01[count_agency:]])
            agency_matrix[count_agency, ] = sim
            count_agency += 1
        agency_dict = dict(zip(range(len(agency_list)), agency_list))
        new_final_dict[group] = pd.DataFrame(agency_matrix).rename(columns = agency_dict, index = agency_dict)
        
        
    return new_final_dict 


In [204]:
six_final_dict = get_new_matrix(transactions)

Agricultural services
Airlines
Amusement and entertainment
Business services
Car rental
Clothing outlets
Contracted services
Government services
Lodging
Miscellaneous outlets
Professional services and membership organizations
Repair services
Retail outlets
Service providers
Transportation
Utilities
nan


In [112]:
# Bug Fix 

final_dict_new = {}

for j in final_dict: 
    
    shape_df = final_dict[j].shape
    
    new_df = final_dict[j]/5 + np.diag([0.8 for x in range(shape_df[0])])
    
    final_dict_new[j] = new_df
    
    

In [212]:
final_dict_all = {}
for x in final_dict_new:
    final_dict_all[x] = (six_final_dict[x] + 5* final_dict_new[x])/6
    



,Alcoholic Beverage Regulation Administration,Board of Elections & Ethics,Board of Real Property Assessment & Appeals,Child & Family Services Administration,City Administrator/Deputy Mayor,Commission of Judicial Disabilities & Tenure,Commission on Arts & Humanities,Criminal Justice Coordinating Council,DC BD OF ETHICS AND GOVT ACCOUNTABILITY,DC Council,...,Office of the Mayor,Office of the Secretary,Office of the State Superintendent of Education,Office on Aging,Office on Asian/Pacific Affairs,Public Employee Relations Board,Public Service Commission,Serve DC,University of the District of Columbia,Unknown New Agency
Alcoholic Beverage Regulation Administration,1.000000,0.983185,0.716797,0.762227,0.900299,0.989054,0.957692,0.958787,0.956582,0.947463,...,0.971483,0.987766,0.645254,0.935295,0.960557,0.952669,0.867452,0.968474,0.784269,0.740836
Board of Elections & Ethics,0.983185,1.000000,0.699982,0.751000,0.888871,0.982886,0.974303,0.950184,0.957246,0.964074,...,0.966328,0.988177,0.633719,0.945561,0.977168,0.961614,0.851249,0.951659,0.773321,0.742236
Board of Real Property Assessment & Appeals,0.716797,0.699982,1.000000,0.605300,0.714834,0.710882,0.680715,0.707628,0.731228,0.671522,...,0.698693,0.710850,0.506806,0.693542,0.687899,0.715739,0.681802,0.727847,0.684014,0.623945
Child & Family Services Administration,0.762227,0.751000,0.605300,1.000000,0.861928,0.768113,0.750999,0.800815,0.793754,0.770534,...,0.784672,0.762822,0.881555,0.780715,0.746791,0.785998,0.884596,0.793531,0.835537,0.831976
City Administrator/Deputy Mayor,0.900299,0.888871,0.714834,0.861928,1.000000,0.905984,0.888870,0.938687,0.931625,0.906783,...,0.922543,0.900694,0.744848,0.918586,0.884662,0.923869,0.945099,0.929012,0.881505,0.838310
Commission of Judicial Disabilities & Tenure,0.989054,0.982886,0.710882,0.768113,0.905984,1.000000,0.962820,0.967298,0.963548,0.958409,...,0.974628,0.988812,0.650833,0.941199,0.963688,0.956217,0.868362,0.968773,0.790435,0.750026
Commission on Arts & Humanities,0.957692,0.974303,0.680715,0.750999,0.888870,0.962820,1.000000,0.950184,0.946928,0.979077,...,0.966101,0.965786,0.633719,0.969578,0.984890,0.945831,0.851248,0.941637,0.773320,0.742236
Criminal Justice Coordinating Council,0.958787,0.950184,0.707628,0.800815,0.938687,0.967298,0.950184,1.000000,0.967134,0.963049,...,0.980246,0.962007,0.683535,0.952111,0.945975,0.950951,0.901064,0.977154,0.820708,0.779859
DC BD OF ETHICS AND GOVT ACCOUNTABILITY,0.956582,0.957246,0.731228,0.793754,0.931625,0.963548,0.946928,0.967134,1.000000,0.940294,...,0.959086,0.968816,0.676473,0.959930,0.952321,0.975150,0.894003,0.976842,0.816075,0.784254
DC Council,0.947463,0.964074,0.671522,0.770534,0.906783,0.958409,0.979077,0.963049,0.940294,1.000000,...,0.967111,0.955557,0.653254,0.958033,0.967709,0.935780,0.870783,0.942708,0.788768,0.757921


In [213]:
# Save sim matrices

try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('sim_matrix_all.p', 'wb') as fp:
    pickle.dump(final_dict_all, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
    


In [214]:
# Open Sim matrices
with open('sim_matrix_all.p', 'rb') as fp:
    sim_matrix_open = pickle.load(fp)

In [181]:
def jmp_rank_vendors(similarity_vector, data, group = None, mcc = None):
    '''
    similarity_vector must be indexed by agency name!!! :*
    '''
    rank_dict = defaultdict(int)
    if group: data = data[data["GROUP"] == group]
    if mcc: data = data[data["join_key"] == mcc]
    existing_relationships = data.groupby('AGENCY')['VENDOR_NAME'].apply(set).to_dict()
    for agency in similarity_vector.index :
        for vendor in existing_relationships[agency]:
            rank_dict[vendor] += similarity_vector[agency]*1 #this is where we could 
            
    
    return {k: v for k, v in sorted(rank_dict.items(), key=lambda item: item[1], reverse = True)}



In [182]:
def get_recommendations(agency, group):

    print(jmp_rank_vendors(final_dict_new[agency][group], transactions, 
                    group, ))

{'UNITED    01670619896864', 'UNITED    01670619689524', 'ROADPOST USA', 'SOUTHWES  5262412113196', 'UNITED    0164514060726', 'USAIRWAY  03724687807046', 'USAIRWAY  03724615605270', 'USAIRWAY  03724243786826', 'USAIRWAY  03724243786605', 'ROADPOST USA INC.', 'USAIRWAY  03724687806420', 'UNITED    01670868603423', 'AMERICAN AIRLINES', 'USAIRWAY  03724271044480', 'FRONTIER  42221718359040', 'DELTA     00623612925950', 'SOUTHWEST', 'VERIZON WRLS 72133-01', 'AMERICAN  00123055636320', 'DELTA     00623090873790', 'UNITED    01677402630864', 'DELTA     00623489954765', 'CONTINEN  00521784710716', 'UNITED    01677402630875', 'AIRTRANAI 33270504436013', 'UNITED    0164513848819', 'FRONTIER  S9JR9J', 'USAIRWAY  03724274774032', 'USAIRWAY  03724284698886', 'UNITED    01670628289444', 'USAIRWAY  03787037198044', 'DELTA', 'UNITED    01623303626906', 'TELE-TOWN HALL, LLC', 'UNITED    01623203732393', 'AMERICAN  00123674872210', 'AIRTRANAI 33201170915700'}


{'SOUTHWEST': 52.80206850018226,
 'AMERICAN AIRLINES': 51.092884492358735,
 'DELTA': 48.94685737276934,
 'UNITED AIRLINES': 46.24270031555602,
 'JETBLUE   27': 31.761407232532118,
 'ALASKA A  02': 30.915551882092608,
 'HILTON HOTELS': 22.009305793658257,
 'SPIRIT AIRLINES': 17.232442397936595,
 'SMART CITY NETWORKS': 15.862355161489756,
 'AIR CANADA': 9.814531247493509,
 'HILTON HOTEL AUSTIN': 8.271779395799816,
 'VERIZON WRLS 72133-01': 7.477605434726147,
 'HILTON HOTELS CHICAGO': 7.18353995913611,
 'AT&T DATA': 6.705140358890209,
 'SPRINT *WIRELESS': 6.510247992711861,
 'HILTON DISNEY WORLD': 6.010001000342335,
 'RINGCENTRAL, INC': 4.561343258706608,
 'SMART CITY-WIFI SRVS': 4.555296730602419,
 'HILTON ADVPURCH8002367': 4.218433047638012,
 'AVIANCA   13': 4.157261068136446,
 'VZWRLSS*ETM NNYD239301': 3.981617776486912,
 'AT&T K004 9966 ESTOR': 3.860596815253384,
 'TWILIO': 3.837977890551351,
 'TELCOM FXR LLC': 3.4408461948547977,
 'GLOBALSTAR, INC.': 3.3724299529131185,
 'SPRINT WIRE